In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import librosa
import glob
import sys
#import yaml
import pickle

from keras.callbacks import CSVLogger
from keras.optimizers import Adam

sys.path.insert(0,'..')
from mavd.model import build_custom_cnn
from mavd.data_generator_URBAN_SED import DataGenerator
from mavd.callbacks import *

os.environ["CUDA_VISIBLE_DEVICES"]="0"

Using TensorFlow backend.


In [2]:
# files parameters
Nfiles = None
resume = False
load_subset = Nfiles

# audio parameters
sr = 22050
sequence_time = 1.0
sequence_hop_time = 1.0
audio_hop = 512
audio_win = 1024
n_fft = 1024
normalize_data = 'none' # para comparar loss con MST
get_annotations = True
mel_bands = 128
normalize_energy = True

# training
learning_rate = 0.001
epochs = 101
batch_size = 64
sed_early_stopping = 100
epoch_limit = None
fit_verbose = True
fine_tuning = False

#model
large_cnn = True
frames = True

label_list = (['air_conditioner', 'car_horn', 'children_playing',
               'dog_bark', 'drilling', 'engine_idling', 'gun_shot',
               'jackhammer', 'siren', 'street_music'])  

 # Create output folders
expfolder = '../exps/S-CNN_baseline/'


audio_folder = '/data_ssd/users/pzinemanas/maestria/URBAN-SED/audio22050'
#feature_folder = '../../MedleyDB/22050'
label_folder='/data_ssd/users/pzinemanas/maestria/URBAN-SED/annotations'

alpha = 10**8 #REF del log

In [3]:
params = {'files_batch':20, 'path':audio_folder, 'sequence_time': sequence_time, 'sequence_hop_time':sequence_hop_time,'label_list':label_list,'alpha': alpha,'normalize_energy':normalize_energy,
          'audio_hop':audio_hop, 'audio_win':audio_win,'n_fft':n_fft,'sr':sr,'mel_bands':mel_bands,'normalize':normalize_data, 'frames':frames,'get_annotations':get_annotations}

#params['path'] = audio_folder
#params['label_list'] = label_list
#sequence_frames = int(np.ceil(params['sequence_time']*params['sr']/params['audio_hop']))
sequence_frames = int(np.ceil(sequence_time*sr/audio_hop))

# Datasets
partition = {}# IDs
labels = {}# Labels

test_files = sorted(glob.glob(os.path.join(audio_folder,'test', '*.wav')))
val_files = sorted(glob.glob(os.path.join(audio_folder,'validate', '*.wav')))

if load_subset is not None:
    test_files = test_files[:load_subset]
    val_files = val_files[:load_subset]

test_labels = {}
test_mel = {}
val_labels = {}
val_mel = {}
print('Founding scaler')
for n,id in enumerate(test_files):
    labels[id] = os.path.join(label_folder, 'test',os.path.basename(id).replace('.wav','.txt'))
    #train_mel[id]  = os.path.join(mel_folder, 'train',os.path.basename(id).replace('.wav','.npy.gz'))
for id in val_files:
    labels[id] = os.path.join(label_folder, 'validate',os.path.basename(id).replace('.wav','.txt'))

params['train'] = False

# Generators
print('Making generators')
test_generator = DataGenerator(test_files, labels, **params)
#scaler = training_generator.get_scaler()
#print('scaler',scaler)

#params['scaler'] = scaler
#params['train'] = False
params['sequence_hop_time'] = sequence_time

validation_generator = DataGenerator(val_files, labels, **params)

print('Getting data')

_,_,x_val,y_val = validation_generator.return_all()
_,_,x_test,y_test = test_generator.return_all()

print(x_val.shape, y_val.shape)

sequence_frames = x_val.shape[1]

# Build model

print('\nBuilding model...')

sequence_samples = int(sequence_time*sr)

model = build_custom_cnn(n_freq_cnn=mel_bands, n_frames_cnn=sequence_frames,large_cnn=large_cnn)

model.summary()

weights_best_file = os.path.join(expfolder, 'weights_best.hdf5')
model.load_weights(weights_best_file)

# Fit model
print('\nTesting model...')

y_test_predicted = model.predict(x_test)
y_val_predicted = model.predict(x_val)

#np.save('predict_proba.npy',y_val_predicted)
#np.save('test_proba.npy',y_val)


#np.save(os.path.join(expfolder, 'y_test_predict.npy'),y_test_predicted)
#np.save(os.path.join(expfolder, 'y_test.npy'),y_test)

data = {'val': {'y': y_val, 'predicted': y_val_predicted}, 'test': {'y': y_test, 'predicted': y_test_predicted}}
filename = os.path.join(expfolder,'results.pickle')
pickle.dump(data, open(filename, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

print(y_test.shape)

print(F1(y_test,y_test_predicted))
print(ER(y_test,y_test_predicted))
print(F1(y_val,y_val_predicted))
print(ER(y_val,y_val_predicted))

Founding scaler
Making generators
Getting data
(20000, 44, 1024)
(20000, 44, 1024)


W0829 11:32:35.150956 139965999163136 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0829 11:32:35.160333 139965999163136 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0829 11:32:35.203613 139965999163136 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0829 11:32:35.221370 139965999163136 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecate

(20000, 44, 128) (20000, 10)

Building model...


W0829 11:32:36.395885 139965999163136 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0829 11:32:36.636306 139965999163136 deprecation.py:506] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 44, 128)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 44, 128)           512       
_________________________________________________________________
lambda_1 (Lambda)            (None, 44, 128, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 124, 128)      3328      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 62, 128)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 20, 62, 128)       512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 58, 128)       409728    
__________